# Applied Machine Learning Systems (MLS-1) - ELEC0134

# Final Assignment - Classification of Tumours

## Task A - Binary Classification using Support Vector Machines (SVM)

## Completed by Student Number - 18014580

This is a Jupyter Notebook submitted as part of the final assignment for the Applied Machine Learning Systems (MLS-1) coursework which involves tumor classification and identification.

This particular notebook tests the SVM method using the SVC model from the sklearn library on Task A to find the accuracy of using support vector machines classification for this particular task.

### Importing different packages

Initially, we must import the different packages needed for this task. The packages required to implement suport vector machines for this binary task can be found in pandas, sklearn (scikit-learn), skimage (scikit-image), and numpy. Please ensure these are installed in your virtualenv before running this code.

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score

from skimage import io
from skimage.io import imread, imshow
from skimage.color import rgb2gray

# import tensorflow as tf
#from keras.preprocessing import image
import matplotlib.pyplot as plt

import os
from os import listdir

from datetime import datetime

print("All imports carried out successfully")

All imports carried out successfully


### Loading the data from the data sets

In this cell we are using read_csv to load the labels for each image into a dataframe. We then display the dataframe to ensure the data from the .csv file was loaded correctly.

In [3]:
#Loading the CSV file 'label' containing the different labels for each MRI Scan

tumour_labels = pd.read_csv('./dataset/label.csv')
print(tumour_labels.shape) #Outputs array with the shape of the dataframe to ensure all images, filenames and labels loaded

(3000, 2)


In [4]:
tumour_labels #Displaying data frame to visually ensure data loaded correctly

,file_name,label
0,IMAGE_0000.jpg,tumor
1,IMAGE_0001.jpg,no_tumor
2,IMAGE_0002.jpg,tumor
3,IMAGE_0003.jpg,tumor
4,IMAGE_0004.jpg,tumor
...,...,...
2995,IMAGE_2995.jpg,no_tumor
2996,IMAGE_2996.jpg,tumor
2997,IMAGE_2997.jpg,tumor
2998,IMAGE_2998.jpg,tumor


### Printing out dataset to ensure labels changed accordingly

This is a binary classification task, so in the cells below, we modify the data frame by locating all the images whose label is not "no_tumor" and setting them to tumor rather than having them as the types of tumor.

In [5]:
#change anything in the label column that is not equal to "no_tumor" to "tumor" rather than the type of tumor
tumour_labels.loc[tumour_labels['label']!= 'no_tumor', 'label'] = 'tumor' 

In [1]:
tumour_labels #Displaying data frame to visually ensure labels changed as specified in cell above

NameError: name 'tumour_labels' is not defined

### Adding numeric representation to the data frame¶

In the cells below, we add another column to the data frame and assign a numeric value label to each label. Since our problem is binary, the numeric label can only be 1 or 0.

If the label indicates "tumor" it is assigned a numeric label 1, and if the label does not indicate a tumour "no_tumor" it is assigned the numeric label 0. It also increases adaptability or convenience of the code if deep-learning models were to be used to have numeric label rather than a text label.

In [ ]:
classes = ['no_tumor', 'tumor']
for CLASS in classes:
    tumour_labels.loc[tumour_labels['label'] == CLASS, 'numeric label'] = classes.index(CLASS)

In [ ]:
tumour_labels #Displaying dataframe to ensure new column created and values are being assigned properly

### Choosing mode and loading the images

In the cell below, we give the user the option to select whether or not to convert the images from RGB to greyscale as their could be different in runtime and accuracy. Any such results will be reported in the written report. Below are some average metrics for RGB and Greyscale.

In the runs carried out, the average metrics for each are:

RGB (Accuracy, Time Taken ...)
Grayscale (Accuracy, Time Taken ... )

In [ ]:
print('As default, this program loads images with their RGB channels. Would you like to change the images to grayscale? \n')
color_mode = input('Please type in Yes or No: ')

if color_mode.lower().startswith("y"):
    print("\n You selected Yes. The images will be loaded as Grayscale images.")
elif color_mode.lower().startswith("n"):
    print("\n You selected No. The images will be loaded with their RGB channels.")
else:
    print("\n Invalid choice. Please select Yes or No.")
    color_mode = input('\n Please type in Yes or No: ')

In the cell below, we load the image files using the imread functions from the scikit-image package. We convert the images to numpy arrays before appending them to a list. This list of arrays is then converted to a numpy array of arrays to allow us to print the shape to verify all the images have been loaded.

This cell also uses datetime.now() to output the time elapsed to load the image set. This was used to compare efficiency of different imread functions from different packages including skimage, cv2 etc. In the end, skimage was chosen as it produced slightly favourable results.

In [6]:
#Uncomment to verify contents of the directory loaded

#dirname = './dataset/image'
#print(listdir(dirname))

#Code used to test timing

start = datetime.now() #Stores the time at which loading starts

images = [] #An empty list is created

#If statement uses user input color_mode to decide whether to convert images to greyscale or not before populating list
#The for loops reads all images from the directory as listdir allows it loop through name of all files in the directory

if color_mode.lower().startswith("n"):
    for file in listdir('./dataset/image'): 
        img = io.imread('./dataset/image/' + file) 
        img = np.array(img) #Converting image loaded to a numpy array
        images.append(img) #Appending the img as a numpy array into the list images
        
        #This can be uncommented if we want to show all the images but data is too large so we usually don't
        #plt.figure()
        #plt.imshow(img)
        
elif color_mode.lower().startswith("y"):
    for file in listdir('./dataset/image'):
        img = io.imread('./dataset/image/' + file)
        greyimg = rgb2gray(img) #Converting each image to grayscale 
        greyimg = np.array(greyimg) #Converting image loaded to a numpy array
        images.append(greyimg) #Appending the img as a numpy array into the list images
        
        #plt.figure()
        #plt.imshow(img)
    
#Marking the end of the code
end = datetime.now()
elapsed = end - start #Calculating time elapsed while comparing different packages used to load images
print('The time elapsed to load the images was: ', + elapsed) 

#The red colormap applied will only appear if it is a grayscalew image. The image is chosen randomly to be images[45]
io.imshow(np.array(images[45]), cmap='Reds') #Verifying visually if the image is greyscale or not by applying red colormap

print(np.array(images).shape) #Converting list images to a numpy array before outputting shape to verify 

(3000, 512, 512, 3)


### Splitting the data into training and testing data

In the cells below, we use train_test_split from the model_selection part of the sklearn package. This splits the data into training and testing data before we train the model. The testing data is left unseen so we can test our trained model using it.

In [7]:
X = np.array(images) #Set X to contain the array of images
Y = tumour_labels['numeric label'] #Set Y to contain the corresponding numeric labels of each image

xTrain, xTest, yTrain, yTest = train_test_split(X,Y, random_state = 0) #Set random_state to 0 for reproducibility

print(xTrain.shape) #Printing the shape of xTrain to veryify data was split correctly
print(xTest.shape) #Printing the shape of xTest to verify  data was split correctly
train_size = xTrain.shape[0]
test_size = xTest.shape[0]

(2250, 512, 512, 3)
(750, 512, 512, 3)


### Reshaping xTrain and xTest for use in ML models and verifying the reshape

These cells reshape xTrain and xTest to 2-dimensional arrays and verifies they have been correctly reshaped depending on whether or not they were converted to grayscale.

In [24]:
#Reshaping xTrain and xTest each to 2-dimensional arrays to allow them to be input into models such as Logistic Reg and SVMs

#If statement to control reshaping of xTrain and xTest dependent if using RGB or Grayscale
#If RGB then .shape arrays will have 4 elements, so we reshape using [1], [2] and [3] from xTrain.shape
#Otherwise, if Grayscale .shape arrays will only have 3 elements so we reshape using only [1] and [2] as there is no [3]

if len(xTrain.shape) == 4:
    xTrain = xTrain.reshape(xTrain.shape[0], xTrain.shape[1]*xTrain.shape[2]*xTrain.shape[3])
    xTest = xTest.reshape(xTest.shape[0], xTest.shape[1]*xTest.shape[2]*xTest.shape[3])
elif len(xTrain.shape) == 3:
    xTrain = xTrain.reshape(xTrain.shape[0], xTrain.shape[1]*xTrain.shape[2])
    xTest = xTest.reshape(xTest.shape[0], xTest.shape[1]*xTest.shape[2])    

In [25]:
xTrain.shape #Visually verifying xTrain has been reshaped

(2250, 786432)

In [26]:
xTest.shape #Visually verifying xTest has been reshaped

(750, 786432)

In [27]:
if color_mode.lower().startswith("n"):
    if xTrain.shape[0] == train_size and xTrain.shape[1] == 512*512*3:
        print('xTrain reshaped correctly')
    else: print('xTrain reshaped incorrectly')
    if xTest.shape[0] == test_size and xTest.shape[1] == 512*512*3:
        print('xTest reshaped correctly')
    else: print('xTest reshaped incorrectly')
elif color_mode.lower().startswith("y"):
    if xTrain.shape[0] == train_size and xTrain.shape[1] == 512*512:
        print('xTrain reshaped correctly')
    else: print('xTrain reshaped incorrectly')
    if xTest.shape[0] == test_size and xTest.shape[1] == 512*512:
        print('xTest reshaped correctly')
    else: print('xTest reshaped incorrectly')

xTrain reshaped correctly
xTest reshaped correctly


### Training

In [10]:
model1_svmlinear = SVC(kernel='linear')
#model2_svmpoly
#model3_svmrbf
model.fit(xTrain, yTrain)


In [ ]:
#y_Pred = LogRegClassification(xTrain, yTrain, xTest)
y_Pred = model.predict(xTest)
print('Accuracy on test set: '+str(accuracy_score(yTest,y_Pred)))
print(classification_report(yTest,y_Pred))#text report showing the main classification metrics

### Testing and reporting new accuracy score using updated test set uploaded on XX Dec

### Plot the confusion matrix and then show which images were misclassified and see if they are the same each time

### See if any specific techniques can be used to help with the images

### Training loss, validation loss and hyperpara tuning